<a href="https://colab.research.google.com/github/navidh86/perturbseq-10701/blob/master/Enformer_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install enformer-pytorch

In [5]:
from enformer_pytorch import Enformer
import torch
import numpy as np

In [6]:
# ============================
# 1. Load the Enformer model
# ============================
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load pretrained model
model = Enformer.from_pretrained('EleutherAI/enformer-official-rough').to(device)
model.eval()
print("Model loaded successfully!")




config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Model loaded successfully!


In [12]:
# ============================
# 2. Generate a dummy DNA sequence
# ============================
SEQ_LEN = 393216  # Enformer expects this length
nucleotides = np.array(["A", "C", "G", "T"])
dummy_seq = "".join(np.random.choice(nucleotides, SEQ_LEN))
print(f"Dummy sequence length: {len(dummy_seq)}")



Dummy sequence length: 393216


In [17]:
# ============================
# 3. Tokenize / encode sequence
# ============================

# One-hot encode
mapping = {"A":0, "C":1, "G":2, "T":3}
oh = np.zeros((1, SEQ_LEN, 4), dtype=np.float32)
for i, base in enumerate(dummy_seq):
    oh[0, i, mapping[base]] = 1.0
x = torch.from_numpy(oh).to(device)


In [18]:
# ============================
# 4. Forward pass through Enformer
# ============================
# Run forward with embeddings
with torch.no_grad():
    out = model(x, return_embeddings=True)


In [22]:
# ============================
# 5. Extract embeddings
# ============================

preds, embeds = out  # unpack the tuple

print("Available prediction heads:", preds.keys())  # e.g., 'human', 'mouse'
print("Human prediction shape:", preds['human'].shape)
print("Embeddings shape:", embeds.shape)

# Optionally, pool embeddings
pooled = embeds.mean(dim=1)
print("Pooled embedding shape:", pooled.shape)

Available prediction heads: dict_keys(['human', 'mouse'])
Human prediction shape: torch.Size([1, 896, 5313])
Embeddings shape: torch.Size([1, 896, 3072])
Pooled embedding shape: torch.Size([1, 3072])


In [23]:

print(embeds[0, :2, :5])

tensor([[-5.2886e-02, -9.6655e-07, -1.3949e-01, -1.6360e-01, -1.5532e-01],
        [-3.2889e-02, -5.9410e-07, -1.3008e-01, -1.5920e-01, -1.5800e-01]],
       device='cuda:0')
